In [1]:
#hide
#skip
%config Completer.use_jedi = False
# upgrade fastrl on colab
! [ -e /content ] && pip install -Uqq fastrl['dev'] pyvirtualdisplay && \
                     apt-get install -y xvfb python-opengl > /dev/null 2>&1 
# NOTE: IF YOU SEE VERSION ERRORS, IT IS SAFE TO IGNORE THEM. COLAB IS BEHIND IN SOME OF THE PACKAGE VERSIONS

In [2]:
# hide
from fastcore.imports import in_colab
# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev.showdoc import *
    from nbdev.imports import *
    if not os.environ.get("IN_TEST", None):
        assert IN_NOTEBOOK
        assert not IN_COLAB
        assert IN_IPYTHON
else:
    # Virutual display is needed for colab
    from pyvirtualdisplay import Display
    display = Display(visible=0, size=(400, 300))
    display.start()

In [3]:
# default_exp agents.dqn.core

In [4]:
# export
# Python native modules
import os
from collections import deque
# Third party libs
import torch
from torch.nn import *
from fastcore.all import *
from fastai.learner import *
from fastai.torch_basics import *
from fastai.torch_core import *
from fastai.callback.all import *
# Local modules
from fastrl.data.block import *
from fastrl.data.gym import *
from fastrl.agent import *
from fastrl.core import *
from fastrl.memory.experience_replay import *

/opt/conda/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  /opt/conda/conda-bld/pytorch_1616554793803/work/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


# DQN Core
> Contains the minimum DQN API.

In [5]:
# export
class DQN(Module):
    def __init__(self,state_sz:int,action_sz:int,hidden=512):
        self.layers=Sequential(
            Linear(state_sz,hidden),
            ReLU(),
            Linear(hidden,action_sz),
        )
    def forward(self,x): return self.layers(x)

In [6]:
dqn=DQN(4,2)
dqn(torch.randn((10,4)))

tensor([[-0.4468,  0.1879],
        [-0.3948,  0.2509],
        [-0.2506,  0.1942],
        [-0.4980,  0.3435],
        [-0.2752,  0.0451],
        [-0.8294,  0.7163],
        [ 0.2236,  0.2529],
        [-0.2411, -0.0111],
        [-0.5606,  0.2089],
        [-0.1297,  0.1693]], grad_fn=<AddmmBackward>)

In [7]:
# export
class ArgMaxFeed(AgentCallback):
    def before_action(self): 
        raw_action=self.agent.model(self.experience['state'].to(default_device()))
        self.agent.raw_action_shape=raw_action.shape
        self.agent.action=torch.argmax(raw_action,dim=1).reshape(-1,1)
        
class DiscreteEpsilonRandomSelect(AgentCallback):
    
    def __init__(self,epsilon=0.5,idx=0,min_epsilon=0.2,max_epsilon=1,max_steps=5000):
        store_attr()
    
    def before_noise(self): 
        self.mask=torch.randn(size=(self.agent.action.shape[0],))<self.epsilon
        self.experience['randomly_selected']=self.mask.reshape(-1,1)
        self.experience['epsilon']=torch.full(self.agent.action.shape,self.epsilon)
        self.experience['orignal_actions']=self.agent.action.detach().clone()
        self.agent.action[self.mask]=self.agent.action[self.mask].random_(0,self.agent.raw_action_shape[1])
        self.agent.action=self.agent.action.detach().cpu().numpy()
    
        if self.agent.model.training: 
            self.idx+=1
            self.epsilon=max(self.min_epsilon,self.max_epsilon-self.idx/self.max_steps)

In [8]:
agent=Agent(dqn,cbs=[ArgMaxFeed,DiscreteEpsilonRandomSelect(max_epsilon=0.5)]).to(default_device())
agent.do_action(state=torch.randn((10,4)).to(default_device()))


(array([[0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [1],
        [0]]),
 {'state': tensor([[ 0.0196,  1.4058,  1.5048,  0.4740],
          [ 0.2119, -0.3726, -0.7019, -0.8312],
          [ 0.6107, -1.8964,  1.2379, -1.8776],
          [ 1.8259, -0.7774,  0.0058,  0.4619],
          [ 0.5262,  0.0362,  0.4679, -0.4745],
          [-0.0352,  0.8968, -2.1272, -0.2465],
          [-0.7233,  0.4747,  1.0834, -0.4030],
          [-0.4129, -0.1818,  1.5259,  1.0423],
          [-1.1912,  0.3462,  0.2864,  2.7926],
          [ 1.4336,  1.6439, -0.8415, -0.3937]]),
  'randomly_selected': tensor([[ True],
          [False],
          [ True],
          [ True],
          [ True],
          [False],
          [False],
          [False],
          [ True],
          [ True]]),
  'epsilon': tensor([[0.5000],
          [0.5000],
          [0.5000],
          [0.5000],
          [0.5000],
          [0.5000],
          [0.5000],
          [0.

In [9]:
agent.show_loop()

Start Action
   - before_preprocess: []
   - after_preprocess: []
   - before_action  : [ArgMaxFeed]
   - after_action   : []
   - before_noise   : [DiscreteEpsilonRandomSelect]
   - after_noise    : []
End Action


In [10]:
# export        
class Epsilon(Metric):
    order=30
    epsilon=0

    @property
    def value(self): return self.epsilon
    def reset(self): self.epsilon=0
    def accumulate(self,learn):
        for cb in learn.model.cbs:
            if type(cb)==DiscreteEpsilonRandomSelect:
                self.epsilon=cb.epsilon

In [11]:
# # export
# class ExperienceReplay(Callback):
#     def __init__(self,*args,bs=16,max_sz=200,warmup_sz=100,**kwargs):
#         store_attr()
#         self.memory=None
#         self.pointer=0
    
#     def after_pred(self):
#         xb=BD(self.learn.xb[0]).mapv(to_detach)
#         if self.memory is None:            self.memory=xb
#         elif self.memory.bs()<self.max_sz: self.memory+=xb
#         else:
#             self.memory=self.memory[:self.pointer]+xb+self.memory[self.pointer+xb.bs():]
#             self.pointer+=xb.bs()
#             if self.pointer>self.max_sz: self.pointer=0
#         with torch.no_grad():
#             idxs=np.random.randint(0,self.memory.bs(),self.bs).tolist()
#             self.learn.xb=(self.memory[idxs].mapv(to_device),)
        
#         if self.memory.bs()<self.warmup_sz: raise CancelBatchException

In [12]:
# export
class DQNTrainer(Callback):
    "Performs traditional training on `next_q`. Requires a callback such as `RegularNextQ`"
    def __init__(self,discount=0.99,n_steps=1):
        store_attr()
        self._xb=None  
    
    def after_pred(self): 
        self.learn.yb=self.xb
        self.learn.xb=self.xb[0]
        self._xb=({k:v.clone() for k,v in self.xb.items()},)
        self.learn.done_mask=self.xb['done'].reshape(-1,)
        self.learn.next_q=self.learn.model.model(self.xb['next_state']).max(dim=1).values.reshape(-1,1)
        self.learn.next_q[self.done_mask]=0 #xb[done_mask]['reward']
        self.learn.targets=self.xb['reward']+self.learn.next_q*(self.discount**self.n_steps)
        self.learn.pred=self.learn.model.model(self.xb['state'])
        t_q=self.pred.clone()
        t_q.scatter_(1,self.xb['action'],self.targets)
        # finalize the xb and yb
        self.learn.yb=(t_q,)
        
    def before_backward(self): 
        self.learn.xb=self._xb

In [13]:
# slow
dqn=DQN(4,2)
agent=Agent(dqn,cbs=[ArgMaxFeed,DiscreteEpsilonRandomSelect])
source=Source(cbs=[GymLoop('CartPole-v1',agent,steps_count=3,seed=0,
                           steps_delta=1,),FirstLast])

dls=SourceDataBlock().dataloaders([source],n=1000,bs=1,num_workers=0)

exp_replay=ExperienceReplayCallback(bs=32,max_sz=100000,warmup_sz=32)

learn=Learner(dls,agent,loss_func=MSELoss(),
              cbs=[exp_replay,DQNTrainer],
              metrics=[Reward,Epsilon,NEpisodes])

Could not do one pass in your dataloader, there is something wrong in it


In [15]:
slow=False
learn.fit(50 if slow else 1,lr=0.0001,wd=0)

> Note: This should work without a exp replay although will perform poorly.

In [ ]:
# hide
from fastcore.imports import in_colab

# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev.export import *
    from nbdev.export2html import *
    from nbdev.cli import make_readme
    make_readme()
    notebook2script()
    notebook2html()